In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define paths and file names 
target_path = 'Targets/daily_crsp.csv'

firm_charac_path = 'Predictors/CompFirmCharac.csv'
jkp_path = 'Predictors/jkp.csv'
#earnings_path = 'Predictors/earnings_calls.parquet'
#mda_path = 'Predictors/mda_text.parquet'

link_table_path = 'linking_table.csv'


In [3]:
# Start with limited number of rows
daily_data = pd.read_csv(target_path, nrows = 100000)

In [4]:
daily_data

,PERMNO,HdrCUSIP,CUSIP,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,DlyCalDt,DlyRet,sprtrn
0,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-03,0.007353,-0.009549
1,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-04,-0.014599,-0.038345
2,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-05,0.014815,0.001922
3,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-06,-0.007299,0.000956
4,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-07,-0.007353,0.027090
...,...,...,...,...,...,...,...,...,...,...,...
99995,10119,90130N10,90130N10,TW,NaN,8064,6331,0,2000-01-31,-0.019672,0.025218
99996,10119,90130N10,90130N10,TW,NaN,8064,6331,0,2000-02-01,0.016722,0.010628
99997,10119,90130N10,90130N10,TW,NaN,8064,6331,0,2000-02-02,0.009868,-0.000114
99998,10119,90130N10,90130N10,TW,NaN,8064,6331,0,2000-02-03,0.003257,0.011248


In [5]:
daily_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   PERMNO         100000 non-null  int64  
 1   HdrCUSIP       100000 non-null  object 
 2   CUSIP          99976 non-null   object 
 3   Ticker         92797 non-null   object 
 4   TradingSymbol  89279 non-null   object 
 5   PERMCO         100000 non-null  int64  
 6   SICCD          100000 non-null  int64  
 7   NAICS          100000 non-null  int64  
 8   DlyCalDt       100000 non-null  object 
 9   DlyRet         92605 non-null   float64
 10  sprtrn         100000 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 8.4+ MB


In [6]:
daily_data.describe()

,PERMNO,PERMCO,SICCD,NAICS,DlyRet,sprtrn
count,100000.000000,100000.000000,100000.000000,100000.000000,92605.000000,100000.000000
mean,10060.055740,10302.603960,5209.398760,271971.266500,0.000716,0.000195
std,35.835979,8861.269373,2086.039956,222573.981654,0.044040,0.012437
min,10001.000000,1728.000000,0.000000,0.000000,-0.642857,-0.119841
25%,10028.000000,7976.000000,3570.000000,0.000000,-0.012048,-0.005435
50%,10051.000000,7992.000000,5730.000000,325412.000000,0.000000,0.000489
75%,10100.000000,8045.000000,7370.000000,511210.000000,0.011686,0.006065
max,10119.000000,53202.000000,8300.000000,711110.000000,2.226667,0.115800


In [7]:
# DATA SANTIZATION 
daily_data.dropna(subset=['PERMNO', 'DlyCalDt', 'DlyRet'], inplace=True)          # Drop rows containing NaN returns or PERMNO
daily_data['DlyCalDt'] = pd.to_datetime(daily_data['DlyCalDt'])                  # Convert dates to datetime format
daily_data.sort_values(by=['PERMNO', 'DlyCalDt'], inplace=True)                   # Sort the dataset by PERMNO and date
daily_data.drop_duplicates(subset=['PERMNO', 'DlyCalDt'], inplace=True)           # Drop any duplicate entries
daily_data.drop(columns=['CUSIP', 'HdrCUSIP', 'TradingSymbol'], inplace=True)     # Drop unused columns

daily_data.describe()

,PERMNO,PERMCO,SICCD,NAICS,DlyCalDt,DlyRet,sprtrn
count,92550.000000,92550.000000,92550.000000,92550.000000,92550,92550.000000,92550.000000
mean,10058.734554,10566.000832,5125.636153,290383.519806,2009-10-12 15:35:44.596433920,0.000717,0.000188
min,10001.000000,1728.000000,0.000000,0.000000,2000-01-03 00:00:00,-0.642857,-0.119841
25%,10028.000000,7978.000000,3570.000000,0.000000,2003-06-05 00:00:00,-0.012061,-0.005509
50%,10051.000000,7999.000000,5094.000000,326113.000000,2008-07-07 00:00:00,0.000000,0.000487
75%,10100.000000,8045.000000,6794.000000,511210.000000,2015-04-28 00:00:00,0.011695,0.006111
max,10119.000000,53202.000000,8300.000000,711110.000000,2024-12-31 00:00:00,2.226667,0.115800
std,36.314466,9151.196265,2069.305746,218678.568326,NaN,0.044052,0.012514


In [8]:
comp = pd.read_csv(firm_charac_path, nrows = 100000, low_memory=False)

In [9]:
comp

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsqpy,xoptepsy,xoptqpy,xopty,xrdy,xsgay,exchg,cik,costat,fic
0,1000,1966-03-31,1966,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,I,USA
1,1000,1966-06-30,1966,2.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,I,USA
2,1000,1966-09-30,1966,3.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,I,USA
3,1000,1966-12-31,1966,4.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,I,USA
4,1000,1967-03-31,1967,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,I,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2620,2017-12-31,2017,4.0,12,INDL,C,D,STD,0176A,...,NaN,NaN,NaN,NaN,NaN,2852.0,0,1011659.0,A,USA
99996,2620,2018-03-31,2018,1.0,12,INDL,C,D,STD,0176A,...,NaN,NaN,NaN,NaN,NaN,760.0,0,1011659.0,A,USA
99997,2620,2018-06-30,2018,2.0,12,INDL,C,D,STD,0176A,...,NaN,NaN,NaN,NaN,NaN,1523.0,0,1011659.0,A,USA
99998,2620,2018-09-30,2018,3.0,12,INDL,C,D,STD,0176A,...,NaN,NaN,NaN,NaN,NaN,2264.0,0,1011659.0,A,USA


In [10]:
missing_counts = comp.isna().sum()
print(missing_counts.describe())


count       256.000000
mean      76605.558594
std       29176.695759
min           0.000000
25%       57501.500000
50%       88373.500000
75%       98041.500000
max      100000.000000
dtype: float64


In [28]:
# Convert datadate to datetime
comp['datadate'] = pd.to_datetime(comp['datadate'])

# Remove duplicates
comp.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)

# Drop rows where gvkey or datadate is missing
comp.dropna(subset=['gvkey', 'datadate'], inplace=True)

# Identify and remove columns with excessive missing data (e.g., > 90%)
missing_threshold = 0.5
valid_cols = comp.columns[comp.isna().mean() < missing_threshold]
comp = comp[valid_cols]

# Optionally convert string-based numeric columns to actual numbers
#for col in comp.select_dtypes(include='object').columns:
#    comp[col] = pd.to_numeric(comp[col], errors='ignore')


/tmp/ipykernel_31367/1311088561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['datadate'] = pd.to_datetime(comp['datadate'])
/tmp/ipykernel_31367/1311088561.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)
/tmp/ipykernel_31367/1311088561.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp.dropna(s

In [29]:
ccm = pd.read_csv(link_table_path)

In [30]:
ccm

,GVKEY,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,LU,25881,23369,1970-11-13,1978-06-30
1,1001,LU,10015,6398,1983-09-20,1986-07-31
2,1002,LC,10023,22159,1972-12-14,1973-06-05
3,1003,LU,10031,6672,1983-12-07,1989-08-16
4,1004,LU,54594,20000,1972-04-24,E
...,...,...,...,...,...,...
33380,352262,LC,23773,59507,2023-03-17,E
33381,353444,LC,23209,59330,2022-07-22,E
33382,355398,LC,25134,59886,2024-05-17,E
33383,356128,LC,24704,59765,2024-01-19,E


In [31]:
# Convert dates
ccm['LINKDT'] = pd.to_datetime(ccm['LINKDT'])

# First, replace 'E' with a far-future date (e.g., end of 2024 or 2099)
ccm['LINKENDDT'] = ccm['LINKENDDT'].replace('E', '2024-12-31')

ccm['LINKENDDT'] = pd.to_datetime(ccm['LINKENDDT'])
ccm['LINKENDDT'].fillna(pd.to_datetime('2024-12-31'), inplace=True)

# Merge Compustat with link table
ccm.rename(columns={'GVKEY': 'gvkey'}, inplace=True)
comp_linked = pd.merge(comp, ccm, how='left', on='gvkey')

# Filter out invalid links (only use LN (link to security), LS, etc.)
valid_links = comp_linked['LINKTYPE'].isin(['LU', 'LC'])
comp_linked = comp_linked[valid_links]

# Keep only rows where datadate is within link period
comp_linked = comp_linked[
    (comp_linked['datadate'] >= comp_linked['LINKDT']) &
    (comp_linked['datadate'] <= comp_linked['LINKENDDT'])
]


/tmp/ipykernel_31367/3902899045.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ccm['LINKENDDT'].fillna(pd.to_datetime('2024-12-31'), inplace=True)


In [32]:
comp_linked

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xidocy,exchg,cik,costat,fic,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
19,1000,1970-12-31,1970,4.0,12,INDL,C,D,STD,AE.2,...,NaN,12,NaN,I,USA,LU,25881.0,23369.0,1970-11-13,1978-06-30
20,1000,1971-03-31,1971,1.0,12,INDL,C,D,STD,AE.2,...,NaN,12,NaN,I,USA,LU,25881.0,23369.0,1970-11-13,1978-06-30
21,1000,1971-06-30,1971,2.0,12,INDL,C,D,STD,AE.2,...,NaN,12,NaN,I,USA,LU,25881.0,23369.0,1970-11-13,1978-06-30
22,1000,1971-09-30,1971,3.0,12,INDL,C,D,STD,AE.2,...,NaN,12,NaN,I,USA,LU,25881.0,23369.0,1970-11-13,1978-06-30
23,1000,1971-12-31,1971,4.0,12,INDL,C,D,STD,AE.2,...,NaN,12,NaN,I,USA,LU,25881.0,23369.0,1970-11-13,1978-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138101,2620,2007-09-30,2007,3.0,12,INDL,C,D,STD,0176A,...,-8.118,0,1011659.0,A,USA,LC,20694.0,841.0,1983-12-30,2008-11-03
138102,2620,2007-12-31,2007,4.0,12,INDL,C,D,STD,0176A,...,66.429,0,1011659.0,A,USA,LC,20694.0,841.0,1983-12-30,2008-11-03
138103,2620,2008-03-31,2008,1.0,12,INDL,C,D,STD,0176A,...,-2.225,0,1011659.0,A,USA,LC,20694.0,841.0,1983-12-30,2008-11-03
138104,2620,2008-06-30,2008,2.0,12,INDL,C,D,STD,0176A,...,84.721,0,1011659.0,A,USA,LC,20694.0,841.0,1983-12-30,2008-11-03


In [33]:
# Rename for merge clarity
comp_linked.rename(columns={'datadate': 'quarter_date', 'LPERMNO': 'PERMNO'}, inplace=True)
comp_linked['PERMNO'] = comp_linked['PERMNO'].astype('int64')

# Merge daily returns with fundamentals using PERMNO
merged = pd.merge_asof(
    daily_data.sort_values('DlyCalDt'),
    comp_linked.sort_values('quarter_date'),
    by='PERMNO',
    left_on='DlyCalDt',
    right_on='quarter_date',
    direction='backward'
)


In [34]:
merged

,PERMNO,Ticker,PERMCO,SICCD,NAICS,DlyCalDt,DlyRet,sprtrn,gvkey,quarter_date,...,sstky,xidocy,exchg,cik,costat,fic,LINKTYPE,LPERMCO,LINKDT,LINKENDDT
0,10001,EWST,7953,4920,0,2000-01-03,0.007353,-0.009549,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,10114,AKLM,9831,7810,0,2000-01-03,0.085366,-0.009549,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,10108,SDS,8049,7374,0,2000-01-03,0.042105,-0.009549,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,10107,MSFT,8048,7370,0,2000-01-03,-0.001606,-0.009549,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,10104,ORCL,8045,7370,0,2000-01-03,0.054099,-0.009549,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92545,10044,RMCF,7992,2060,445292,2024-12-31,-0.004098,-0.004285,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
92546,10028,ELA,7978,5094,423940,2024-12-31,-0.001391,-0.004285,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
92547,10026,JJSF,7976,2052,311821,2024-12-31,-0.001095,-0.004285,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
92548,10066,FKWL,6331,7373,517112,2024-12-31,0.006160,-0.004285,1008.0,1986-11-30,...,0.003,0.0,13.0,NaN,I,USA,LC,6331.0,1983-08-25,1987-02-26


In [35]:
jkp = pd.read_csv(jkp_path, nrows = 100000)

In [36]:
jkp

,location,name,freq,weighting,direction,n_stocks,n_stocks_min,date,ret
0,usa,age,monthly,vw_cap,-1,502,8,1926-03-31,-0.086738
1,usa,age,monthly,vw_cap,-1,505,12,1926-04-30,-0.000572
2,usa,age,monthly,vw_cap,-1,510,17,1926-05-31,-0.006332
3,usa,age,monthly,vw_cap,-1,512,21,1926-06-30,-0.003012
4,usa,age,monthly,vw_cap,-1,503,23,1926-07-31,0.003211
...,...,...,...,...,...,...,...,...,...
99995,usa,rd_me,monthly,vw_cap,1,1854,826,1987-02-28,0.022074
99996,usa,rd_me,monthly,vw_cap,1,1850,841,1987-03-31,-0.015272
99997,usa,rd_me,monthly,vw_cap,1,1844,837,1987-04-30,0.049291
99998,usa,rd_me,monthly,vw_cap,1,1877,873,1987-05-31,-0.002783


In [37]:
# Parse date column
jkp['date'] = pd.to_datetime(jkp['date'])

# Pivot to wide format: one column per factor
jkp_wide = jkp.pivot(index='date', columns='name', values='ret').reset_index()

jkp_wide

name,date,age,aliq_at,aliq_mat,ami_126d,at_be,at_gr1,at_me,at_turnover,be_gr1a,...,pi_nix,ppeinv_gr1a,prc,prc_highprc_252d,qmj,qmj_growth,qmj_prof,qmj_safety,rd5_at,rd_me
0,1926-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.010740,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1926-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.019043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1926-03-31,-0.086738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.048628,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1926-04-30,-0.000572,NaN,NaN,-0.001413,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.001448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1926-05-31,-0.006332,NaN,NaN,-0.003118,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.008217,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,2024-08-31,-0.009159,0.010798,0.018197,-0.035742,-0.027581,-0.009432,-0.012035,-0.008847,0.004118,...,-0.005098,0.009275,-0.027720,0.035217,0.010192,-0.005674,0.014741,0.012970,0.009868,NaN
1184,2024-09-30,0.015388,0.005373,0.007651,-0.009591,-0.007449,0.003045,-0.009871,0.003660,0.008696,...,0.006355,0.008296,-0.006779,0.003206,-0.006422,-0.002243,0.002634,-0.018560,-0.017818,NaN
1185,2024-10-31,0.030176,-0.017033,-0.009476,0.002699,-0.020468,-0.015150,0.006831,-0.030677,0.002992,...,0.003082,0.002214,0.012406,0.021771,-0.006040,0.002241,-0.015539,0.002140,0.024123,NaN
1186,2024-11-30,0.055421,-0.058225,-0.021419,0.017654,-0.017568,-0.029310,0.003207,-0.008216,-0.014642,...,0.001879,0.006882,0.007938,0.016876,-0.005569,0.011715,-0.017477,-0.023964,0.019916,NaN


In [38]:
# Create a month-end date column in your daily return dataset
merged['month_end'] = merged['DlyCalDt'] + pd.offsets.MonthEnd(0)

merged = merged.merge(jkp_wide, left_on='month_end', right_on='date', how='left')

# Optional: drop redundant 'date' column after merge
merged.drop(columns=['date'], inplace=True)

merged

,PERMNO,Ticker,PERMCO,SICCD,NAICS,DlyCalDt,DlyRet,sprtrn,gvkey,quarter_date,...,pi_nix,ppeinv_gr1a,prc,prc_highprc_252d,qmj,qmj_growth,qmj_prof,qmj_safety,rd5_at,rd_me
0,10001,EWST,7953,4920,0,2000-01-03,0.007353,-0.009549,NaN,NaT,...,-0.037852,-0.008723,0.029981,0.004210,-0.040257,-0.035010,-0.057493,-0.000004,0.139700,NaN
1,10114,AKLM,9831,7810,0,2000-01-03,0.085366,-0.009549,NaN,NaT,...,-0.037852,-0.008723,0.029981,0.004210,-0.040257,-0.035010,-0.057493,-0.000004,0.139700,NaN
2,10108,SDS,8049,7374,0,2000-01-03,0.042105,-0.009549,NaN,NaT,...,-0.037852,-0.008723,0.029981,0.004210,-0.040257,-0.035010,-0.057493,-0.000004,0.139700,NaN
3,10107,MSFT,8048,7370,0,2000-01-03,-0.001606,-0.009549,NaN,NaT,...,-0.037852,-0.008723,0.029981,0.004210,-0.040257,-0.035010,-0.057493,-0.000004,0.139700,NaN
4,10104,ORCL,8045,7370,0,2000-01-03,0.054099,-0.009549,NaN,NaT,...,-0.037852,-0.008723,0.029981,0.004210,-0.040257,-0.035010,-0.057493,-0.000004,0.139700,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92545,10044,RMCF,7992,2060,445292,2024-12-31,-0.004098,-0.004285,NaN,NaT,...,-0.015670,0.004686,0.010377,-0.000785,0.002292,-0.006214,0.014735,-0.006817,0.028227,NaN
92546,10028,ELA,7978,5094,423940,2024-12-31,-0.001391,-0.004285,NaN,NaT,...,-0.015670,0.004686,0.010377,-0.000785,0.002292,-0.006214,0.014735,-0.006817,0.028227,NaN
92547,10026,JJSF,7976,2052,311821,2024-12-31,-0.001095,-0.004285,NaN,NaT,...,-0.015670,0.004686,0.010377,-0.000785,0.002292,-0.006214,0.014735,-0.006817,0.028227,NaN
92548,10066,FKWL,6331,7373,517112,2024-12-31,0.006160,-0.004285,1008.0,1986-11-30,...,-0.015670,0.004686,0.010377,-0.000785,0.002292,-0.006214,0.014735,-0.006817,0.028227,NaN


In [ ]:
# Construct lagged returns for predictions
df['DlyRet_lag1'] = df.groupby('PERMNO')['DlyRet'].shift(1)
df['sprtrn_lag1'] = df.groupby('PERMNO')['sprtrn'].shift(1)
